# DFT atom

In [1]:
using Pkg;
Pkg.activate("../../SchrodingerEquationSolver");
include("../src/SchrodingerEquationSolver.jl");
import .SchrodingerEquationSolver as ses
import .ses.Grids as Grids
import .ses.Potentials as Potentials
import .ses.MathUtils as MathUtils
import .ses.Hydrogen as Hydrogen
import .ses.InitialConditions as InitialConditions
import .ses.OneDSchrodingerEquationSolver as odses
import .ses.OneDPoissonEquationSolver as odpes
import .ses.EigenvalueFinders as EigenvalueFinders
import .ses.AtomBasisSet as AtomBasisSet
import .ses.Density as Density
import .ses.ExchangeCorrelation as ExchangeCorrelation
using Plots

  Activating project at `~/Desktop/physics_coding_projects/julia/SchrodingerEquationSolver`


In [2]:
#Define parameters and produce an exponential grid.
r_max::Float64=50.0;#Max radius of space grid.
Z::Int64=10;#Atomic number, also used as the charge of coulomb potential.

## Exponential grid

In [3]:

#b::Float64=0.000904; #By decreasing b the number of points in the grid increase.
grid= ses.Grids.exponential_grid(r_max, Z)#, b=b);
grid_sqrt= grid.^2.0; 
N= size(grid)[1]; #Number of points in the grid.
print("grid size ", N)

grid size 8057

In [6]:
#Initialization of potentials and energies

#Initializing coulomb potential due to nuclei charge.
V_colu::Vector{Float64}= Potentials.coulomb_potential(Z, grid);
#Initializing basis set data structure
basis= AtomBasisSet.init_atom_basis_set(Z, grid);

In [7]:

for i_orbi in basis.orbitals
    #angular potential for l orbital
    V_angu= Potentials.angular_potential(i_orbi.l, grid);
    #Assemble effective potential.
    V_effe= V_colu .+ V_angu;
    #Energy grid to search for new eigenvalue, search around the previous eigenvalue.
    E_grid= Grids.uniform_grid(i_orbi.E - 0.5*i_orbi.E, i_orbi.E + 0.5*i_orbi.E, 250);
    #Find the energy intervals with potential eigenvalues.
    E_intervals, bad_intervals= EigenvalueFinders.find_eigenvalue_intervals(E_grid, V_effe,
                                                    grid,InitialConditions.atom, l=i_orbi.l);
    #Search for a solution to the independent particle Schrodinger equation for the first energy interval, 
    #there should be only one interval. Any other interval would be of a higher energy.
    u_temp, ei_temp= EigenvalueFinders.illinois_eigenvalue_finder(E_intervals[1], V_effe, grid,InitialConditions.atom);
    #Update eigenvalue and eigenfunction in the basis set data structure.
    i_orbi.E=ei_temp;
    i_orbi.u=u_temp;

end


In [8]:
for i_orbi in basis.orbitals
    println(i_orbi.E)
end

-50.00004423719211
-12.500011059234811
-12.500003686399625


In [6]:
E_total # Z=2 exponential_grid

-2.8348377103009534